In [2]:
# v tomto kode nacitame udaje o pathway z Hetionetu a Reactome a konvertujeme ich 
# do RDF grafu "https://pathway_ontology.org"

# importujeme potrebne kniznice
import json
from rdflib import Namespace, URIRef, Literal, Dataset
from rdflib.namespace import RDF, SKOS

# definujeme namespace pre bioregistry 
BIOR = Namespace("https://bioregistry.io/")

# nacitanie JSON suboru
with open('/Users/simonsnopko/Desktop/Škola/bakalarska praca/hetionet-v1.0.json') as f:
    data = json.load(f)
    
# nacitame txt subor
with open("/Users/simonsnopko/Desktop/Škola/bakalarska praca/ReactomePathways.txt", "r") as f:
    lines = f.readlines()

# vytvorime RDF dataset a priradime mu meno
ds = Dataset()
graph_name = URIRef("http://pathway_ontology.org")
graph = ds.graph(graph_name)

# prechadzame cez vsetky uzly v JSON datach z Hetionetu
for item in data['nodes']:
    # kontrolujeme ci je typ uzla Pathway"
    if item['kind'] == 'Pathway' and 'url' in item['data']:
        # vytvorime URI pre dany uzol v bioregistry
        identifier = item['identifier'].split('_')[0]
        bior_xref = URIRef(f"{BIOR}wikipathways:{identifier}")
        # priradime tripletu s nazvom (prefLabel) a CURIE (notation) do grafu
        graph.add((bior_xref, SKOS.notation, Literal("wikipathways:" + identifier)))
        graph.add((bior_xref, SKOS.prefLabel, Literal(item['name'])))
            
# prechadzame cez vsetky riadky txt suboru Reactome
for line in lines:
    # rozdelime riadok na jednotlive casti (stlpce) podla tabulatora
    parts = line.strip().split("\t")   
    # ak je pocet casti (stlpcov) 3
    if len(parts) == 3:
        # priradime jednotlive casti k premennym
        pathway_id, pathway_name, organism = parts
        
        # vytvorime URI pre danu pathway v bioregistry
        bior_xref = URIRef(f"{BIOR}reactome:{pathway_id}")
        
        # pridame tripletu s nazvom (prefLabel) a notaciou (notation) pre danu pathway do grafu
        graph.add((bior_xref, SKOS.prefLabel, Literal(pathway_name)))
        graph.add((bior_xref, SKOS.notation, Literal("reactome:" + pathway_id)))

       
# ulozime vytvoreny RDF dataset do suboru vo formate nquads
ds.serialize(destination='pathway_created_ontology.nq', format='nquads')


<Graph identifier=N862ee6c728c44d2d8febbf5352dcb767 (<class 'rdflib.graph.Dataset'>)>